In [1]:
import cadquery as cq

from jupyter_cadquery import (
    versions,
    show, PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer,
    get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay
''
enable_replay(False)

set_defaults(
    cad_width=640, 
    height=480, 
)

import sys
sys.path.append('../')
from config import Config, Shape
from plate import get_base, get_screw_positions, get_key_positions, make_plate
from case import get_basic_shape,  cut_aviator_connector_hole, get_distance_between_two_vertices,  add_microcontrollerbox

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay


/home/ben/anaconda3/envs/jcq3/lib/python3.10/site-packages/quaternion/calculus.py:310: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def fd_indefinite_integral(f, t):


100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.00s


CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [2]:
def make_controller_box_top_plate(case,config):

    controllerBoxLength =  get_distance_between_two_vertices(case.faces(tag='controllerBoxTop').vertices('<X'))['y']
    controllerBoxWidth = get_distance_between_two_vertices(case.faces(tag='controllerBoxTop').vertices('<Y'))['x']

    controller_box_top = (case
                        .faces(tag='controllerBoxTop')
                        .workplane(centerOption="CenterOfBoundBox", invert=True)
                        .move(0,config.wallThickness)
                        .box(controllerBoxWidth,controllerBoxLength, config.controllerBoxThickness, centered=[True,True,False],combine=False)
    )


    return controller_box_top

In [2]:

rkn = [4,8,8,6]
config=Config(rkn, shape=Shape.LEAN, floorThickness=3)
kp = get_key_positions(config)

In [7]:


if True:
    controllerBoxLength =  get_distance_between_two_vertices(case.faces(tag='controllerBoxTop').vertices('<X'))['y']
    controllerBoxWidth = get_distance_between_two_vertices(case.faces(tag='controllerBoxTop').vertices('<Y'))['x']+10

    controller_box_top = (case
                        .faces(tag='controllerBoxTop')
                        .workplane(centerOption="CenterOfBoundBox", invert=True)
                        .move(0,config.wallThickness)
                        .box(controllerBoxWidth,controllerBoxLength, config.controllerBoxThickness, centered=[True,True,False],combine=False)
    )

#case.intersect(controller_box_top)
    


In [9]:
case.faces('+Y',tag='outface').faces('>>Y[-2]').tag('xxxx')


case.faces(tag='xxxx').workplane(centerOption="CenterOfBoundBox").box(60,20,25, centered=[True,True,False])

CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [3]:


def make_case(config:Config) -> cq.Sketch:
    """TODO: add screw holes for microcontroller (check they are in right position!!)
            change fillet parameters
            add module for leds, encoder and aviator
            """
    
    
    case= get_basic_shape(config)
    case.faces('<Z or >Z').tag('bottomface')


    # add box for microcontroller
    if config.controller is not None:
        case = add_microcontrollerbox(case, config)


    #fillet edges
    case = case.edges('|Z').fillet(config.edgeFillet)
    #case.edges('>Z').tag('outerTopEdge')
    case.faces('(not <Z) and (not >Z)').tag('outface')
    case.wires('>Z').all()[0].tag('topwire')

    # scoop out interior
    case= (case.faces(">Z or <Z").shell(-config.wallThickness, kind='intersection'))
    
    # set floor thickness 
    case = case.faces('<Z').wires('>X and >Y').toPending().extrude(config.floorThickness)
    
    return case
    
    #fillet bottom edge
    case = case.edges('<Z').fillet(3)



    # add plate screw holes 
    scr_hls = get_screw_positions(config)
    case=(case.faces('<Z').workplane(origin=(0,0))
          .pushPoints([(x,-y) for x,y in scr_hls])
          .cskHole(2.4, 4.8, 82, depth=None)    
          )

    # add controllerbox holes.
    # TODO - remove hardcoding on screw hole size
    if config.controller is not None:
        case = (case.workplaneFromTagged("controllerBox")
                .rect(config.controller.screw_hole_x,
                        config.controller.screw_hole_y- config.wallThickness - config.controllerYOffset - (config.controller.board_dimension_y -config.controller.screw_hole_y)*0.5, 
                        forConstruction=True ,centered=[True,True,True])
                .vertices()
                .cskHole(2.4, 4.8, 82, depth=None))
    
    #cut hole for aviator connector 
    case = cut_aviator_connector_hole(case, config)

    return case


In [4]:
case = make_case(config)

In [113]:
points = case.edges(cq.NearestToPointSelector(pp[0].toTuple())).vertices().vals()


debug(case.points[0])

NameError: name 'debug' is not defined

In [119]:
vvv= list(filter(lambda x: points[0] in x.Vertices(), case.wires('>Z').vals()[0].Edges()))

#case.wires('>Z').vals()[0].Vertices()[0]


case

vvv[0]

CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [158]:
case.faces('-Y').faces('>>Y').edges('>Z').tag('topPoints')

top_edge_of_box = case.edges(tag='topPoints').val()
vertices_of_box = [v.toTuple() for v in top_edge_of_box.Vertices()]

# find wire with edge in 

wires_to_search = case.wires('>Z').vals()


wire = list(filter(lambda x: top_edge_of_box in x.Edges(), wires_to_search))[0]


#filter wires connected to top edge of box

wanted_edges=[]
for e in wire.Edges():
    start_end_points = [v.toTuple() for v in e.Vertices()]
    if vertices_of_box[0] in start_end_points or  vertices_of_box[1] in start_end_points:
        wanted_edges.append(e)

    

In [159]:
case.newObject(wanted_edges)

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.00s


CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…